# 모델 매개변수 최적화하기

## 기본 코드

In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor())

test_data = datasets.FashionMNIST(root='data', download=True, train=False, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

26422272it [00:22, 1184726.82it/s]                              


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



29696it [00:00, 117266.30it/s]                          


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



4422656it [00:14, 302645.20it/s]                             


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



6144it [00:00, 3082881.18it/s]          


Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



C:\Users\jj950\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 하이퍼 파라미터

In [2]:
learning_rate = 1e-3
batch_size = 64
epoch = 5

## 최적화 단계

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x,y) in enumerate(dataloader):
        pred = model(x)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(x)
            print(f"loss: {loss} [{current} / {size}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for x,y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batch
    correct /= size
    print(f'Test error: \n Accuracy: {(100*correct):>.1f}%, Avg loss: {test_loss:>8f} \n')

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for epoch in range(epochs):
    print(f'Epoch {epoch+1} \n---------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1 
---------------------------
loss: 2.29766583442688 [0 / 60000]
loss: 2.2843403816223145 [6400 / 60000]
loss: 2.2705724239349365 [12800 / 60000]
loss: 2.276184320449829 [19200 / 60000]
loss: 2.2399396896362305 [25600 / 60000]
loss: 2.225576877593994 [32000 / 60000]
loss: 2.240018367767334 [38400 / 60000]
loss: 2.2093868255615234 [44800 / 60000]
loss: 2.207151174545288 [51200 / 60000]
loss: 2.1854772567749023 [57600 / 60000]
Test error: 
 Accuracy: 40.3%, Avg loss: 2.172835 

Epoch 2 
---------------------------
loss: 2.1799182891845703 [0 / 60000]
loss: 2.1633386611938477 [6400 / 60000]
loss: 2.1186447143554688 [12800 / 60000]
loss: 2.1428396701812744 [19200 / 60000]
loss: 2.068453073501587 [25600 / 60000]
loss: 2.02836537361145 [32000 / 60000]
loss: 2.061814308166504 [38400 / 60000]
loss: 1.9884768724441528 [44800 / 60000]
loss: 1.9922658205032349 [51200 / 60000]
loss: 1.9330787658691406 [57600 / 60000]
Test error: 
 Accuracy: 50.2%, Avg loss: 1.921228 

Epoch 3 
-------------